# 📊 00.05: RAG 평가 맛보기 

## 💡 내가 만든 RAG가 얼마나 잘 작동하는지 확인해보자!

In [ ]:
# 1. 간단한 정확도 평가
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

# 평가용 데이터셋
eval_questions = [
    {"question": "파이썬 만든 사람?", "expected_keyword": "귀도"},
    {"question": "RAG란 뭐야?", "expected_keyword": "검색과 생성"},
    {"question": "머신러닝 설명해줘", "expected_keyword": "데이터로 패턴"}
]

documents = [
    "파이썬은 1991년 귀도가 만든 프로그래밍 언어입니다.",
    "머신러닝은 데이터로 패턴을 학습하는 기술입니다.",
    "RAG는 검색과 생성을 합친 AI입니다."
]

docs = [Document(page_content=text) for text in documents]
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(docs, embeddings)

print("📋 평가 데이터셋 준비 완료!")

In [ ]:
# 2. 검색 정확도 측정
correct = 0
total = len(eval_questions)

print("🔍 검색 정확도 평가 중...")
for i, item in enumerate(eval_questions, 1):
    question = item["question"]
    expected = item["expected_keyword"]
    
    # 검색 수행
    results = vectorstore.similarity_search(question, k=1)
    retrieved_text = results[0].page_content
    
    # 키워드 포함 여부 확인
    is_correct = expected in retrieved_text
    if is_correct:
        correct += 1
    
    print(f"{i}. '{question}'")
    print(f"   검색결과: {retrieved_text[:30]}...")
    print(f"   정답여부: {'✅' if is_correct else '❌'}")
    print()

accuracy = correct / total * 100
print(f"📊 검색 정확도: {accuracy:.1f}% ({correct}/{total})")

In [ ]:
# 3. 검색 속도 측정
import time

print("⏱️ 검색 속도 평가 중...")
speeds = []
for question_data in eval_questions:
    question = question_data["question"]
    
    start_time = time.time()
    results = vectorstore.similarity_search(question, k=1)
    end_time = time.time()
    
    speed = end_time - start_time
    speeds.append(speed)
    print(f"'{question}': {speed*1000:.1f}ms")

avg_speed = sum(speeds) / len(speeds)
print(f"\n📊 평균 검색 속도: {avg_speed*1000:.1f}ms")

In [ ]:
# 4. 유사도 점수 분포 확인
print("📈 유사도 점수 분포 확인...")
for question_data in eval_questions:
    question = question_data["question"]
    
    # 점수와 함께 검색
    results = vectorstore.similarity_search_with_score(question, k=3)
    print(f"\n'{question}' 검색 결과:")
    for i, (doc, score) in enumerate(results, 1):
        print(f"  {i}위: {doc.page_content[:25]}... (점수: {score:.3f})")

In [ ]:
# 5. 전체 평가 요약
print("🎉 RAG 평가 완료!")
print(f"✅ 검색 정확도: {accuracy:.1f}%")
print(f"✅ 평균 속도: {avg_speed*1000:.1f}ms")
print(f"✅ 문서 수: {len(documents)}개")
print(f"✅ 평가 케이스: {len(eval_questions)}개")
print("💡 이제 본격적인 RAG 실습을 시작할 준비 완료!")